In [ ]:
!pip install xmltodict

In [ ]:
import pandas as pd
import time
import requests
import json
import xmltodict

In [ ]:
# Query the BGG API to get info and stats on board games
base_url = 'https://boardgamegeek.com/xmlapi2/thing?'
args = '&stats=1&marketplace=1&pagesize=100'

results = pd.DataFrame()
games_per_query = 100
errors = []

# 1 - 10000           0, 100
# 10000 - 20000     100, 200
# 20000 - 30000     200, 300

batch = 41
i_start = 100 * batch
i_end = 100 * (batch + 1)

# Fetch first 1000 board games
for i in range (i_start, i_end):
    # Build CSV string of ids
    start = (games_per_query * i) + 1
    end = (games_per_query * (i + 1)) + 1
    ids = ','.join(str(id) for id in list(range(start, end)))

    # Make API call
    try:
        response = requests.get(f"{base_url}id={ids}{args}")
        # Parse XML response
        # pd.read_xml() doesn't work because of the data shape
        parsed = xmltodict.parse(response.content)
        df = pd.read_json(json.dumps(parsed['items']['item']))
        results = pd.concat([results, df])
        print(f'Fetched ids {start} through {end - 1}')
        time.sleep(1)
    except:
        msg = f'Failed to fetch ids {start} through {end - 1}'
        print(msg)
        print(response)
        errors.append(msg)
        errors.append(response)


In [ ]:
# Verify that there are no errors
errors

In [ ]:
# Copy results
df = results.copy()

# Save to CSV without indexes
df.to_csv(f'bgg_stats_{i_start}01-{i_end}00.csv', index=False)